In [1]:
import copy
from enum import Enum
import json
import os
from typing import Any, Dict, List, Union
import yaml
import uuid

import openai
from pydantic import BaseModel, Field, create_model


# 1. Test schema definition

In [2]:
def create_dynamic_enum(name: str, values: List[Any]) -> Enum:
    return Enum(name, {str(v): v for v in values})

class OpenAIResponseFormatBuilder(object):
    @staticmethod
    def build(
        model: BaseModel, name: str = "schema", strict: bool = True
    ) -> Dict[str, Any]:
        schema = model.model_json_schema()
        schema_dict = {
            "type": "json_schema",
            "json_schema": {"name": name, "strict": strict, "schema": schema},
        }
        return schema_dict

def get_dtype(dtype_name: str) -> type:
    if dtype_name == "str":
        dtype = str
    elif dtype_name == "int":
        dtype = int
    elif dtype_name == "float":
        dtype = float
    else:
        raise ValueError("dtype {} not supported".format(dtype_name))
    return dtype


class SchemaModelMaker(object):
    @staticmethod
    def make(specification: Dict[str, Any], name="model") -> BaseModel:
        """
        specification 다음과 같이 정의한다
        {
                "키값": {
                        "dim": 0, ## 0: str, 1: List[str]
                        "dtype": str, ## 스트링이면 type을 찾도록
                        "allowed_values": ["a", "b"], ## For enum val
                        "optional": False ##
                }
        }
        """
        spec_dict = dict()
        for k, v in specification.items():
            dtype = v.get("dtype", str)
            dim = v.get("dim", 0)
            allowed_values = v.get("allowed_values", None)
            default_val = ...

            if isinstance(dtype, str):
                dtype = get_dtype(dtype)

            if allowed_values:
                allowed_values = [dtype(v) for v in allowed_values]
                enum = create_dynamic_enum(f"{k}-enum", allowed_values)
                dtype = enum

            ## Dim
            if dim == 1:
                dtype = List[dtype]

            spec_dict[k] = (dtype, default_val)

        model = create_model(name, __config__={"extra": "forbid"}, **spec_dict)
        return model

In [3]:
information_specification = {
    "background": {
        "dim": 0,
        "dtype": str,
        "optional": False,
    },
    "scope": {
        "dim": 0,
        "dtype": str,
        "optional": False,
    },
    "impact": {
        "dim": 0,
        "dtype": str,
        "optional": False,
    },
}

Instance = SchemaModelMaker.make(information_specification, name="Instance")
instance = Instance(
    background="",
    scope="",
    impact=""
)
print(instance)

background='' scope='' impact=''


In [4]:
# class InformationColumn(Enum):
# 	background="background"
# 	scope="scope"
# 	impact="impact"	

InformationColumn = create_dynamic_enum("InformationColumn", ["background", "scope", "impact"])
[e.value for e in InformationColumn]

['background', 'scope', 'impact']

In [5]:
class InformationColumn(Enum):
	background="background"
	scope="scope"
	impact="impact"	

## Creation
class Instance(BaseModel):
	background: str
	scope: str
	impact: str
	
class InstanceCreation(BaseModel):
	instance: Instance

## Refinement
class RefinementOperationType(Enum):
	edit="edit"
	append="append"
	
class InstanceRefinement(BaseModel):
	operation_type: RefinementOperationType
	uuid: str
	column: InformationColumn
	value: str
	
class InstanceRefinements(BaseModel):
	# background: List[InstanceRefinement]
	# scope: List[InstanceRefinement]
	# impact: List[InstanceRefinement]
	refinements: List[InstanceRefinement]

## Operation
class InstanceOperationType(Enum):
	create="create"
	refine="refine"
	
class InstanceOperation(BaseModel):
	type: InstanceOperationType
	uuid: str
	operation: Union[InstanceCreation, InstanceRefinements]
	
class InstanceOperations(BaseModel):
	operations: List[InstanceOperation]

In [6]:
schema = InstanceOperations.model_json_schema()
schema

{'$defs': {'InformationColumn': {'enum': ['background', 'scope', 'impact'],
   'title': 'InformationColumn',
   'type': 'string'},
  'Instance': {'properties': {'background': {'title': 'Background',
     'type': 'string'},
    'scope': {'title': 'Scope', 'type': 'string'},
    'impact': {'title': 'Impact', 'type': 'string'}},
   'required': ['background', 'scope', 'impact'],
   'title': 'Instance',
   'type': 'object'},
  'InstanceCreation': {'properties': {'instance': {'$ref': '#/$defs/Instance'}},
   'required': ['instance'],
   'title': 'InstanceCreation',
   'type': 'object'},
  'InstanceOperation': {'properties': {'type': {'$ref': '#/$defs/InstanceOperationType'},
    'uuid': {'title': 'Uuid', 'type': 'string'},
    'operation': {'anyOf': [{'$ref': '#/$defs/InstanceCreation'},
      {'$ref': '#/$defs/InstanceRefinements'}],
     'title': 'Operation'}},
   'required': ['type', 'uuid', 'operation'],
   'title': 'InstanceOperation',
   'type': 'object'},
  'InstanceOperationType': {'

# 2. OpenAI call test

In [7]:
SYSTEM_INSTRUCTION = '''## Instance 설명
이 과정을 추출 결과를 업데이트 해나가는 방식으로 추출하려고 합니다.
'Instance'는 정보의 단위 입니다. 같은 정보의 단위는 Instance로 묶도록 합니다.

주어진 기존 instance 추출 결과를 보고 다음 InstanceOperation중 하나를 수행하세요
* create: 기존과 구분되는 새로운 instance 1개를 새로 정의하여 추가합니다
* refine: 수정이 필요한 기존 instance 하나를 uuid 기반으로 선택 후 수정이 필요한 값에 대한 RefinementOperation을 수행하세요

RefinementOperation은 다음과 같습니다
* edit: 기존 값을 수정
    * ex. 중간에 값을 추가하여 기존 값을 수정해야 하는 경우
    * 수정하는 InstanceColumn의 uuid를 꼭 기입하세요
* append: 추가적인 값을 instance 컬럼에 추가
    * uuid를 빈칸으로 기입합니다.

## 정보 추출 성명
다음은 사업의 제안 요청서 파일에서 읽은 문서 파일 파싱 결과와 중간 추출 결과입니다.
파싱 결과는 과제의 개요에 대한 다음 3가지 정보들을 포함할 수 있습니다.
정보 카테고리:
1. 추진 배경 및 필요성 (background)
* 제안 공고를 올리게 된 이유, 배경 상황
* 사업으로 달성하고자 하는 목적 내용 (기대 효과와는 다르다)
* 사업 (추진) 목적도 이런 정보를 담고 있다
* 주로 '추진 배경', '필요성'의 하위 내용들이다
2. 사업 범위 (scope)
* 사업(과업)을 통해서 개발하려고 하는 내용
* 실질적으로 개발하려는 내용
* 주로 '사업 범위', '사업 내용', '추진 내용'과 같은 정보로 언급됨
3. 기대 효과 (impact)
* 사업을 통해서 기대하는 효과

최종적으로 위 정보를 다음 JSON 스키마로 추출하려고 합니다.
스키마:
{{"background": str, "scope": str, "impact": str}

규칙:
* background, scope, impact는 파싱 결과에서 해당 카테고리에 해당되는 내용을 찾아서 모두 추출합니다.
* 이 내용들은 요약하지 않으며 파싱 결과에 사용된 표현들 그대로 추출합니다.
* 필요없는 정보는 추출하지 않고 정보 카테고리로 정의된 관련 내용만 추출하도록 합니다.
* 파싱 결과에서 테이블은 "<|start_table|>..<|end_table|>" 처럼 감싸져 있습니다
* 테이블이 근거자료와 같이 부가적인 정보를 가지는 경우 참고만 하고 굳이 추출하지는 않습니다
* 정보 카테고리에 핵심적인 내용이 있는 경우만 추출합니다. 이때 표의 구조를 고려하여 상위 항목/하위 항목 카테고리화를 살리도록 합니다.
* 사업명, 기간, 예산, 계약 방법등의 내용은 뽑지 않도록 합니다
* background, scope, impact 정보와 같이 'ㅇ'와 '-'등으로 나열된 정보의 경우는 계층을 살려서 'html 리스트 문법'으로 바꿔 주세요 (ex. "<ul><li>...</ul>", "<ol><li>..</ol>")
* 리스트의 계층 (depth) 정보는 ul/ol 태그를 사용하여 상위-하위 항목의 관계를 꼭 살리도록 합니다.
* 새로운 정보를 '계층에 맞게' ul li 아래에 끼워넣어야 한다면 RefinementOperation을 append가 아닌 'edit'을 합니다
* 순서 정보가 필요한 경우만 ol로 표기합니다
* 상위, 하위 항목 내용은 모두 추출 합니다.
상위/하위 정보 예시:
"1. 상위 정보\n2. 상위정보2\n\tㅇ 하위 정보 1\n\tㅇ 하위 정보 2..\n□. 상위정보3"
-> "<ol><li>상위 정보</li><li>상위정보2<ul><li>하위 정보 1</li><li>하위 정보 2</li></ol><ul><li>상위정보3</li></ul>"
* 카테고리에 해당되는 내용이 없다면 스트링을 비워 주세요

## 최종 스키마
최종 다음 스키마대로 추출하세요
{'$defs': {'InformationColumn': {'enum': ['background', 'scope', 'impact'],
'title': 'InformationColumn',
'type': 'string'},
'Instance': {'additionalProperties': False,
'properties': {'background': {'title': 'Background', 'type': 'string'},
'scope': {'title': 'Scope', 'type': 'string'},
'impact': {'title': 'Impact', 'type': 'string'}},
'required': ['background', 'scope', 'impact'],
'title': 'Instance',
'type': 'object'},
'InstanceCreation': {'properties': {'instance': {'$ref': '#/$defs/Instance'}},
'required': ['instance'],
'title': 'InstanceCreation',
'type': 'object'},
'InstanceOperation': {'properties': {'type': {'$ref': '#/$defs/InstanceOperationType'},
'uuid': {'title': 'Uuid', 'type': 'string'},
'operation': {'anyOf': [{'$ref': '#/$defs/InstanceCreation'},
    {'$ref': '#/$defs/InstanceRefinements'}],
    'title': 'Operation'}},
'required': ['type', 'uuid', 'operation'],
'title': 'InstanceOperation',
'type': 'object'},
'InstanceOperationType': {'enum': ['create', 'refine'],
'title': 'InstanceOperationType',
'type': 'string'},
'InstanceRefinement': {'properties': {'operation_type': {'$ref': '#/$defs/RefinementOperationType'},
'column': {'$ref': '#/$defs/InformationColumn'},
'value': {'title': 'Value', 'type': 'string'}},
'required': ['operation_type', 'column', 'value'],
'title': 'InstanceRefinement',
'type': 'object'},
'InstanceRefinements': {'properties': {'refinements': {'items': {'$ref': '#/$defs/InstanceRefinement'},
    'title': 'Refinements',
    'type': 'array'}},
'required': ['refinements'],
'title': 'InstanceRefinements',
'type': 'object'},
'RefinementOperationType': {'enum': ['edit', 'append'],
'title': 'RefinementOperationType',
'type': 'string'}},
'properties': {'operations': {'items': {'$ref': '#/$defs/InstanceOperation'},
'title': 'Operations',
'type': 'array'}},
'required': ['operations'],
'title': 'InstanceOperations',
'type': 'object'}'''

# USER_TEMPLATE = '''공고명: "{bidding_name}"
# 파싱 결과: "{text}"
# 기존 추출 결과: {previous_instances}'''

USER_TEMPLATE = '''파싱 결과:
"{text}"

기존 추출 결과:
{previous_instances}'''

In [8]:
class InstanceColumn(BaseModel):
    uuid: str
    value: str
    
class InformationInstance(BaseModel):
    uuid: str
    background: List[InstanceColumn]
    scope: List[InstanceColumn]
    impact: List[InstanceColumn]

partial_background = InstanceColumn(
    uuid=str(uuid.uuid4()),
    value="<ul><li>통관 후 유통 중인 수입수산물에 대한 관리체계 고도화 필요<ul><li>유통 중인 수입수산물이 안전검사에 불합격하거나 위해사고가 발생된 경우 유통경로를 역추적, 회수하기 위한 통합 모니터링 시스템 구축 필요</li><li>수산물 안전관리의 실효성을 높이기 위하여 선제적인 위해 수산물 차단 및 회수 체계 시스템 필요</li></ul></li><li>최신 기술 트랜드를 반영하여 관계기관 간 협력과 정보공유를 위한 시스템 고도화 필요</li></ul>"
)

partial_scope = InstanceColumn(
    uuid=str(uuid.uuid4()),
    value="<ul><li>수입수산물 유통 영업자 시스템 고도화<ul><li>상위업체 유통이력 등록요청관리 프로그램 개발</li><li>유통업체 ERP 시스템 연동 프로그램 개발</li><li>사용자 편의성 향상을 위한 유통이력 신고기능 고도화</li></ul></li><li>수입수산물 유통이력 업무 시스템 고도화<ul><li>유통이력 실태점검 기능 고도화</li><li>유통이력 모니터링 검색기능 고도화</li><li>위해수산물 유통 검색엔진 프로그램 개발</li><li>유통이력 추적관리 기능 개선</li><li>업무사용자 편의성 향상을 위한 유통이력 추적관리 기능 개선</li></ul></li><li>GIS기반 유통이력 추적관리 모니터링 시스템 구축<ul><li>대시보드 형태 GIS 추적모니터링 프로그램 개발</li><li>GIS 기반 통합검색 기능 개발</li></ul></li><li>온라인 교육용 컨텐츠 관리시스템 구축<ul><li>동영상 재생관리 프로그램 개발</li><li>통계프로그램 개발</li></ul></li><li>유전자원관리 정보시스템 구축</li></ul>"
)

instance = InformationInstance(
    uuid=str(uuid.uuid4()),
    background=[
        partial_background
    ],
    scope=[
        partial_scope
    ],
    impact=[]
)
print(instance)

uuid='055b4762-a2a0-4b2a-91ef-51566cd8471a' background=[InstanceColumn(uuid='87f6a63b-1604-410d-afbb-f3d731727bbf', value='<ul><li>통관 후 유통 중인 수입수산물에 대한 관리체계 고도화 필요<ul><li>유통 중인 수입수산물이 안전검사에 불합격하거나 위해사고가 발생된 경우 유통경로를 역추적, 회수하기 위한 통합 모니터링 시스템 구축 필요</li><li>수산물 안전관리의 실효성을 높이기 위하여 선제적인 위해 수산물 차단 및 회수 체계 시스템 필요</li></ul></li><li>최신 기술 트랜드를 반영하여 관계기관 간 협력과 정보공유를 위한 시스템 고도화 필요</li></ul>')] scope=[InstanceColumn(uuid='5a876c79-ce74-4581-826a-d14d159d94c5', value='<ul><li>수입수산물 유통 영업자 시스템 고도화<ul><li>상위업체 유통이력 등록요청관리 프로그램 개발</li><li>유통업체 ERP 시스템 연동 프로그램 개발</li><li>사용자 편의성 향상을 위한 유통이력 신고기능 고도화</li></ul></li><li>수입수산물 유통이력 업무 시스템 고도화<ul><li>유통이력 실태점검 기능 고도화</li><li>유통이력 모니터링 검색기능 고도화</li><li>위해수산물 유통 검색엔진 프로그램 개발</li><li>유통이력 추적관리 기능 개선</li><li>업무사용자 편의성 향상을 위한 유통이력 추적관리 기능 개선</li></ul></li><li>GIS기반 유통이력 추적관리 모니터링 시스템 구축<ul><li>대시보드 형태 GIS 추적모니터링 프로그램 개발</li><li>GIS 기반 통합검색 기능 개발</li></ul></li><li>온라인 교육용 컨텐츠 관리시스템 구축<ul><li>동영상 재생관리 프로그램 개발</li><li>통계프로그램 개발</li></ul></li><li>유전자원관리 정보시스템 구축</li

In [9]:
previous_instances = [
    instance
]

previous_instances = [
    x.model_dump() for x in previous_instances
]
previous_instances

[{'uuid': '055b4762-a2a0-4b2a-91ef-51566cd8471a',
  'background': [{'uuid': '87f6a63b-1604-410d-afbb-f3d731727bbf',
    'value': '<ul><li>통관 후 유통 중인 수입수산물에 대한 관리체계 고도화 필요<ul><li>유통 중인 수입수산물이 안전검사에 불합격하거나 위해사고가 발생된 경우 유통경로를 역추적, 회수하기 위한 통합 모니터링 시스템 구축 필요</li><li>수산물 안전관리의 실효성을 높이기 위하여 선제적인 위해 수산물 차단 및 회수 체계 시스템 필요</li></ul></li><li>최신 기술 트랜드를 반영하여 관계기관 간 협력과 정보공유를 위한 시스템 고도화 필요</li></ul>'}],
  'scope': [{'uuid': '5a876c79-ce74-4581-826a-d14d159d94c5',
    'value': '<ul><li>수입수산물 유통 영업자 시스템 고도화<ul><li>상위업체 유통이력 등록요청관리 프로그램 개발</li><li>유통업체 ERP 시스템 연동 프로그램 개발</li><li>사용자 편의성 향상을 위한 유통이력 신고기능 고도화</li></ul></li><li>수입수산물 유통이력 업무 시스템 고도화<ul><li>유통이력 실태점검 기능 고도화</li><li>유통이력 모니터링 검색기능 고도화</li><li>위해수산물 유통 검색엔진 프로그램 개발</li><li>유통이력 추적관리 기능 개선</li><li>업무사용자 편의성 향상을 위한 유통이력 추적관리 기능 개선</li></ul></li><li>GIS기반 유통이력 추적관리 모니터링 시스템 구축<ul><li>대시보드 형태 GIS 추적모니터링 프로그램 개발</li><li>GIS 기반 통합검색 기능 개발</li></ul></li><li>온라인 교육용 컨텐츠 관리시스템 구축<ul><li>동영상 재생관리 프로그램 개발</li><li>통계프로그램 개발</li></ul></li><li>유전자원관리 정보시

In [10]:
contents = {
    "previous_instances": json.dumps(previous_instances, ensure_ascii=False)
}
contents

{'previous_instances': '[{"uuid": "055b4762-a2a0-4b2a-91ef-51566cd8471a", "background": [{"uuid": "87f6a63b-1604-410d-afbb-f3d731727bbf", "value": "<ul><li>통관 후 유통 중인 수입수산물에 대한 관리체계 고도화 필요<ul><li>유통 중인 수입수산물이 안전검사에 불합격하거나 위해사고가 발생된 경우 유통경로를 역추적, 회수하기 위한 통합 모니터링 시스템 구축 필요</li><li>수산물 안전관리의 실효성을 높이기 위하여 선제적인 위해 수산물 차단 및 회수 체계 시스템 필요</li></ul></li><li>최신 기술 트랜드를 반영하여 관계기관 간 협력과 정보공유를 위한 시스템 고도화 필요</li></ul>"}], "scope": [{"uuid": "5a876c79-ce74-4581-826a-d14d159d94c5", "value": "<ul><li>수입수산물 유통 영업자 시스템 고도화<ul><li>상위업체 유통이력 등록요청관리 프로그램 개발</li><li>유통업체 ERP 시스템 연동 프로그램 개발</li><li>사용자 편의성 향상을 위한 유통이력 신고기능 고도화</li></ul></li><li>수입수산물 유통이력 업무 시스템 고도화<ul><li>유통이력 실태점검 기능 고도화</li><li>유통이력 모니터링 검색기능 고도화</li><li>위해수산물 유통 검색엔진 프로그램 개발</li><li>유통이력 추적관리 기능 개선</li><li>업무사용자 편의성 향상을 위한 유통이력 추적관리 기능 개선</li></ul></li><li>GIS기반 유통이력 추적관리 모니터링 시스템 구축<ul><li>대시보드 형태 GIS 추적모니터링 프로그램 개발</li><li>GIS 기반 통합검색 기능 개발</li></ul></li><li>온라인 교육용 컨텐츠 관리시스템 구축<ul><li>동영상 재생관리 프로그램 개발</li><li>통계프로그램 개발</li></ul></li><l

# 3. LLM call

In [11]:
from config import get_settings
settings = get_settings()

In [12]:
sample_fname = "rfp_abstract_sample1"
with open(f"samples/{sample_fname}.yaml", "r") as fr:
    sample_dict = yaml.safe_load(fr)

text = sample_dict["text"]

In [13]:
system_msg = copy.deepcopy(SYSTEM_INSTRUCTION)

user_msg = copy.deepcopy(USER_TEMPLATE).format(
    text=text,
    previous_instances=json.dumps(previous_instances, ensure_ascii=False)
)

print(user_msg)

파싱 결과:
"Ⅰ. 사업개요
1. 사업 기본정보
ㅇ 사 업 명 : 수입수산물 유통이력 관리시스템 고도화
ㅇ 사업기간 : 계약 후 240일(SW개발사업 적정사업기간 산정기준에 따른 사업임)
ㅇ 소요예산 : 550,000,000원(부가세 포함)
ㅇ 계약방법 : 제한경쟁입찰(협상에 의한 계약)
2. 추진 배경 및 필요성
ㅇ 통관 후 유통 중인 수입수산물에 대한 관리체계 고도화 필요
- 유통 중인 수입수산물이 안전검사에 불합격하거나 위해사고가 발생된 경우 유통경로를 역추적, 회수하기 위한 통합 모니터링 시스템 구축 필요
- 수산물 안전관리의 실효성을 높이기 위하여 선제적인 위해 수산물 차단 및 회수 체계 시스템 필요
ㅇ 최신 기술 트랜드를 반영하여 관계기관 간 협력과 정보공유를 위한 시스템 고도화 필요
- 최신 기술의 적절한 활용을 통한 시스템 효율성 증대
- 4차산업 혁명 시대에 부응하기 위해 사용자 중심의 정보고유/가치화를 위한 시스템 고도화 필요
3. 사업 범위
ㅇ 수입수산물 유통 영업자 시스템 고도화
- 상위업체 유통이력 등록요청관리 프로그램 개발
유통업체 ERP 시스템 연동 프로그램 개발
사용자 편의성 향상을 위한 유통이력 신고기능 고도화
ㅇ 수입수산물 유통이력 업무 시스템 고도화
- 유통이력 실태점검 기능 고도화
유통이력 모니터링 검색기능 고도화
위해수산물 유통 검색엔진 프로그램 개발
유통이력 추적관리 기능 개선
업무사용자 편의성 향상을 위한 유통이력 추적관리 기능 개선
ㅇ GIS기반 유통이력 추적관리 모니터링 시스템 구축
- 대시보드 형태 GIS 추적모니터링 프로그램 개발
GIS 기반 통합검색 기능 개발
ㅇ 온라인 교육용 컨텐츠 관리시스템 구축
- 동영상 재생관리 프로그램 개발
- 통계프로그램 개발
ㅇ 유전자원관리 정보시스템 구축
- 매년 조사∙갱신되는 수산물에 대한 염기서열 등의 DB구축
- 유전자(염기서열) 정보의 검색 및 데이터베이스 현황 제공"

기존 추출 결과:
[{"uuid": "055b4762-a2a0-4b2a-91ef-51566cd8471a

In [14]:
messages = [
    {"role": "system", "content": system_msg},
    {"role": "user", "content": user_msg},
]
messages

[{'role': 'system',
  'content': '## Instance 설명\n이 과정을 추출 결과를 업데이트 해나가는 방식으로 추출하려고 합니다.\n\'Instance\'는 정보의 단위 입니다. 같은 정보의 단위는 Instance로 묶도록 합니다.\n\n주어진 기존 instance 추출 결과를 보고 다음 InstanceOperation중 하나를 수행하세요\n* create: 기존과 구분되는 새로운 instance 1개를 새로 정의하여 추가합니다\n* refine: 수정이 필요한 기존 instance 하나를 uuid 기반으로 선택 후 수정이 필요한 값에 대한 RefinementOperation을 수행하세요\n\nRefinementOperation은 다음과 같습니다\n* edit: 기존 값을 수정\n    * ex. 중간에 값을 추가하여 기존 값을 수정해야 하는 경우\n    * 수정하는 InstanceColumn의 uuid를 꼭 기입하세요\n* append: 추가적인 값을 instance 컬럼에 추가\n    * uuid를 빈칸으로 기입합니다.\n\n## 정보 추출 성명\n다음은 사업의 제안 요청서 파일에서 읽은 문서 파일 파싱 결과와 중간 추출 결과입니다.\n파싱 결과는 과제의 개요에 대한 다음 3가지 정보들을 포함할 수 있습니다.\n정보 카테고리:\n1. 추진 배경 및 필요성 (background)\n* 제안 공고를 올리게 된 이유, 배경 상황\n* 사업으로 달성하고자 하는 목적 내용 (기대 효과와는 다르다)\n* 사업 (추진) 목적도 이런 정보를 담고 있다\n* 주로 \'추진 배경\', \'필요성\'의 하위 내용들이다\n2. 사업 범위 (scope)\n* 사업(과업)을 통해서 개발하려고 하는 내용\n* 실질적으로 개발하려는 내용\n* 주로 \'사업 범위\', \'사업 내용\', \'추진 내용\'과 같은 정보로 언급됨\n3. 기대 효과 (impact)\n* 사업을 통해서 기대하는 효과\n\n최종적으로 위 정보를 다음 JSON 스키마로 추출하려고 

In [15]:
## call generation
from openai_client import OpenAIClient

In [16]:
client = OpenAIClient()

In [17]:
result = await client.chat_completion_with_json_schema(
    response_format=InstanceOperations,
    messages=messages,
    model="gpt-4o-mini-2024-07-18",
    # model="gpt-4o",
    decode_params={
        "temperature": 0.3,
        "top_p": 1.0,
        # "max_completion_tokens": 16000
    }
)

In [19]:
operations = json.loads(result)
print(json.dumps(operations, indent = "\t", ensure_ascii=False))

{
	"operations": [
		{
			"type": "refine",
			"uuid": "055b4762-a2a0-4b2a-91ef-51566cd8471a",
			"operation": {
				"refinements": [
					{
						"operation_type": "append",
						"uuid": "",
						"column": "impact",
						"value": "<ul><li>4차산업 혁명 시대에 부응하기 위해 사용자 중심의 정보고유/가치화를 위한 시스템 고도화 필요</li></ul>"
					}
				]
			}
		}
	]
}
